In [1]:
# Imports
import re
import pickle
import nltk
import sklearn
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score

In [5]:
from google.colab import files
uploaded = files.upload()

Saving dataset_aula02.csv to dataset_aula02 (1).csv


In [6]:
df = pd.read_csv('dataset_aula02.csv')

In [32]:
df.shape

(50000, 2)

In [33]:
df.head()

,review,sentiment
0,one review mention watch 1 oz episod hook righ...,1
1,wonder littl product br br film techniqu unass...,1
2,thought wonder way spend time hot summer weeke...,1
3,basic famili littl boy jake think zombi closet...,negative
4,petter mattei love time money visual stun film...,1


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [35]:
df.sentiment.value_counts()

,count
sentiment,
1,25000
negative,25000


In [36]:
#ajusta labels para representação numerica
df.sentiment.replace('positive', 1, inplace = True)
df.sentiment.replace('negative', 0, inplace = True)

<ipython-input-36-c81fe4744de7>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.sentiment.replace('positive', 1, inplace = True)
<ipython-input-36-c81fe4744de7>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [38]:
#funcao de limpeza geral de dados
def limpa_dados(texto):
	cleaned = re.compile(r'<.*?')
	return re.sub(cleaned, '', texto)

In [39]:
#testar funcao
texto_com_tags = "<p>Este é um exemplo <b>com</b> tags HTML.</p>"
texto_limpo = limpa_dados(texto_com_tags)
print(texto_limpo)

p>Este é um exemplo b>com/b> tags HTML./p>


In [40]:
#aplica funcao ao nosso dataset
df.review = df.review.apply(limpa_dados)

In [41]:
#retira caracteres alfanumerico
def limpa_caracter_especial(texto):
    rem = ''
    for i in texto:
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
    return rem

In [42]:
#testando a funcao
texto_com_caracteres_especiais = "olá, mundo! Como vai?"
texto_limpo = limpa_caracter_especial(texto_com_caracteres_especiais)
print(texto_limpo)

olá  mundo  Como vai 


In [45]:
df.review = df.review .apply(limpa_caracter_especial)

In [44]:
def converte_minusculo(texto):
    return texto.lower()
frase = "Esta é uma frasE com LETRAS MaiúscuLAs"
frase_saida = converte_minusculo(frase)
print(frase_saida)

esta é uma frase com letras maiúsculas


In [20]:
df.review = df.review.apply(converte_minusculo)

In [21]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [27]:
def remove_stopwords(texto):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(str(texto))
    return [w for w in words if w not in stop_words]

In [28]:
frase = 'They are right, as this is exactly what happedned with me.'
frase_saida = remove_stopwords(frase)

print(frase_saida)

['They', 'right', ',', 'exactly', 'happedned', '.']


In [46]:
df.review = df.review.apply(remove_stopwords)

In [30]:
def stemmer(texto):
    objeto_stemmer = SnowballStemmer('english')
    return " ".join([objeto_stemmer.stem(w) for w in texto])

In [31]:
df.review = df.review.apply(stemmer)